In [1]:
# Jupyter notebook related
%reload_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
from dataset import EvoNetV1Dataset

evo_ds = EvoNetV1Dataset()
feats, target, latlon = evo_ds[0]
feats.shape, target.shape, latlon.shape

/home/dzanaga/miniconda3/envs/eo/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/dzanaga/miniconda3/envs/eo/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any 

(torch.Size([36, 128, 128]),
 torch.Size([9, 128, 128]),
 torch.Size([2, 128, 128]))

In [ ]:
from dataset import get_train_val_dataloaders

train_loader, val_loader, train_dataset, val_dataset = (
    get_train_val_dataloaders("configs/base_mep_debug.json")
)

/home/dzanaga/miniconda3/envs/eo/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
batch = train_loader.__iter__().__next__()

2024-10-30 23:16:58.926 | ERROR    | dataset:__getitem__:355 - Error reading evo target for location 54JYQ_092_36
2024-10-30 23:16:59.912 | ERROR    | dataset:__getitem__:355 - Error reading evo target for location 53HPD_083_46
2024-10-30 23:17:00.420 | ERROR    | dataset:__getitem__:355 - Error reading evo target for location 55JBH_089_61
2024-10-30 23:17:00.717 | ERROR    | dataset:__getitem__:355 - Error reading evo target for location 54HTJ_056_36
/data/users/Private/dzanaga/repos/evotrain/src/evotrain/train/lcm10/v002/dataset.py:219: RuntimeWarning: invalid value encountered in divide
  probs = probs / probs.sum(axis=0, keepdims=True)
2024-10-30 23:17:01.358 | ERROR    | dataset:__getitem__:355 - Error reading evo target for location 55JEF_049_35
2024-10-30 23:17:02.683 | ERROR    | dataset:__getitem__:355 - Error reading evo target for location 55JBL_070_46
2024-10-30 23:17:02.756 | ERROR    | dataset:__getitem__:355 - Error reading evo target for location 59GLJ_001_08
2024-10-30

In [5]:
batch[0].shape, batch[1].shape, batch[2].shape

(torch.Size([32, 36, 128, 128]),
 torch.Size([32, 9, 128, 128]),
 torch.Size([32, 2, 128, 128]))

In [14]:
locs = train_dataset.evo_dataset.locs
loc_id = "59GLJ_001_08"
locs[locs.location_id == loc_id]

,location_id,lab_10,lab_20,lab_30,lab_40,lab_50,lab_60,lab_70,lab_80,lab_90,...,dominant,mix_ratio,lon,lat,cell_id,h3_index,group,status,user_alias,test
11,59GLJ_001_08,0.854309,0.001099,0.144287,0.0,0.0,0.000305,0.0,0.0,0.0,...,10,0.205926,168.555613,-46.044709,E170S45,82da87fffffffff,1,SUBMITTED,Andrii (External),False


In [ ]:
from satio_pc import parallelize

reader = evo_ds.evo_dataset._reader()


def check(loc_id):
    fn = reader._annotation_patch_path(loc_id, 2020, "evo-v1")
    return fn.exists()


valid_flag = parallelize(check, locs.location_id.values, max_workers=20)

  0%|          | 0/14381 [00:00<?, ?it/s]

In [6]:
# test inference preview

from callbacks import InferencePreview